# Uber Exercise - Group 2

In [4]:
import requests
import time #this will prevent us from DDOSing their systems
import json
import pandas as pd

In [9]:
def uberPrices(auth_token, start_latitude, start_longitude, end_latitude, end_longitude, seat_count):   
    with requests.session() as c:
        url = 'https://api.uber.com/v1.2/estimates/price'
        query = {'start_latitude':  start_latitude, 'start_longitude': start_longitude, 'end_latitude': end_latitude, 'end_longitude': end_longitude, 'seat_count': seat_count}
        headers = {'Accept-Language': 'en_US', 'Content-Type': 'application/json', 'Authorization': 'Token ' + auth_token }
        time.sleep(.5)
        r = requests.get(url, params=query, headers=headers)
        print r.status_code, r.reason, ' - ', r.url
        return r

def import_response(response, key):
    response_as_dict = json.loads(response.content)
    if key in response_as_dict:
        return pd.read_json(json.dumps(response_as_dict[key]))
    else:
        return []

In [11]:
startingPoint = {"GA": [38.90487,-77.034028]}

customer_list = {"Cheri": [39.025115,-77.103353], "Justin":[38.906454,-77.045986], "Dave":[38.93254,-77.03848],"Sabanat":[38.948455,-77.065242],"Reed":[38.918398,-77.023228],"Jia":[38.927664,-77.053913]}
merged_list = startingPoint.copy()
merged_list.update(customer_list)

#make it a series to ensure iteration order since python's dict is unordered. it doesn't keep order upon iterate, weird.
persons = pd.Series(merged_list, index=['GA','Cheri', 'Justin', 'Dave', 'Sabanat', 'Reed', 'Jia']) 

count = 0 
countNext = 1
maxLen = len(persons)

df_uberXL = pd.DataFrame()

#loop through for uberXL results
for person, location in persons.iteritems():
    
    if count < maxLen:
        if countNext < maxLen-1:
            countNext = count + 1
            name = list(persons.keys())[countNext] #list to pull out names easily by index
            response = uberPrices('PCHhQEbECrygbOjQzsHIwysH3oriDGpxa_WayfrX', persons[count][0], persons[count][1], persons[countNext][0], persons[countNext][1], 2)
            
            df = import_response(response, 'prices')
            df['customer_name'] = name
            df_uberXL = pd.concat([df_uberXL, df[df['display_name'] == 'uberXL']]) #just include uberXL estimate
        
        count = int(count) + 1
    
df_uberXL.reset_index(inplace=True, drop=True)
df_uberXL

200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.034028&end_longitude=-77.103353&start_latitude=38.90487&end_latitude=39.025115&seat_count=2
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.103353&end_longitude=-77.045986&start_latitude=39.025115&end_latitude=38.906454&seat_count=2
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.045986&end_longitude=-77.03848&start_latitude=38.906454&end_latitude=38.93254&seat_count=2
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.03848&end_longitude=-77.065242&start_latitude=38.93254&end_latitude=38.948455&seat_count=2
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.065242&end_longitude=-77.023228&start_latitude=38.948455&end_latitude=38.918398&seat_count=2
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.023228&end_longitude=-77.053913&start_latitude=38.918398&end_latitude=38.927664&seat_count=2


,currency_code,display_name,distance,duration,estimate,high_estimate,localized_display_name,low_estimate,product_id,customer_name
0,USD,uberXL,10.68,1560,$27-34,34.0,uberXL,27.0,9ffa937e-7d2e-4bcf-bc2b-ffec4ef24380,Cheri
1,USD,uberXL,9.77,1320,$25-32,32.0,uberXL,25.0,9ffa937e-7d2e-4bcf-bc2b-ffec4ef24380,Justin
2,USD,uberXL,2.14,480,$9-12,12.0,uberXL,9.0,9ffa937e-7d2e-4bcf-bc2b-ffec4ef24380,Dave
3,USD,uberXL,2.08,420,$8-11,11.0,uberXL,8.0,9ffa937e-7d2e-4bcf-bc2b-ffec4ef24380,Sabanat
4,USD,uberXL,3.70,660,$12-16,16.0,uberXL,12.0,9ffa937e-7d2e-4bcf-bc2b-ffec4ef24380,Reed
5,USD,uberXL,2.31,600,$10-13,13.0,uberXL,10.0,9ffa937e-7d2e-4bcf-bc2b-ffec4ef24380,Jia
